# Generating Genome and Injecting SNPs

## 1. Generate Genes

In [2]:
# python imports
from random import choice
import random

# Biopython imports
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import DNAAlphabet
from Bio import SeqIO
import glob

def createFrag(size):
    DNA = ""
    for count in range(size * 1000):
        DNA += choice("CGTA")
    return DNA

def get_haps(heterozygous_rate, size_kB):
    gene = createFrag(size_kB)
    alpha = set(['A', 'C', 'G', 'T'])
    records = []

    s_len = len(gene)
    s = gene
    delta = round(s_len * heterozygous_rate)
    edit_pos = random.sample(range(0, s_len), delta)
    sl = list(gene)

    
    for edit in edit_pos:
        o = sl[edit]
        new_alpha = alpha - set([o])
        new_char = list(new_alpha)[random.randint(0,2)]
        sl[edit] = new_char
        

    s1 = gene
    s2 = "".join(sl)
    
    return s1, s2
    

def generate_sequences(project_name, get_seq):
    ![ -e $project_name/genome_data ] && rm -r $project_name/genome_data
    !mkdir -p $project_name/genome_data
    
    s1, s2 = get_seq()
    
    records = []
    record = SeqRecord(Seq(s1, DNAAlphabet), id='sample_gen_h1', description="hap 1")    
    records.append(record)
    record = SeqRecord(Seq(s2, DNAAlphabet), id="sample_gen_h2", description="hap 2")    
    records.append(record)
    SeqIO.write(records, project_name + "/genome_data/complete_genome.fa", "fasta")
    for rec in records:
        SeqIO.write(rec, project_name + "/genome_data/" + rec.id + ".fasta", "fasta")
    
    # record diff positions
    file = open(project_name + '/genome_data/diff_pos.txt', 'w+')
    for edit in range(len(s1)):
        if s1[edit] != s2[edit]: file.write(str(edit + 1) + '\n')
    file.close()
    
    return s1, s2

## 2. Simulate Reads

In [3]:
def generate_reads(project_name):
    ![ -e $project_name/reads ] && rm -r $project_name/reads
    !mkdir -p $project_name/reads
    !cd $project_name/reads && simlord \
    --read-reference ../genome_data/complete_genome.fa \
    --coverage 50 \
    --probability-threshold 0.0001 \
    -pi 0.0001 \
    -pd 0.0001 \
    -ps 0.0001 \
    --fixed-readlength 25000 \
    --without-ns sd

    !cat $project_name/reads/sd*.fastq > $project_name/reads/all.fastq

# Checking validity of SNP calling and haplotype phasing

## 1. SNP calling and Phasing

In [4]:
def snp_and_phase(project_name, ref_hap_path):
    ![ -e $project_name/mappings ] && rm -r $project_name/mappings
    ![ -e $project_name/snps ] && rm -r $project_name/snps
    ![ ! -e $project_name/mappings ] && mkdir -p $project_name/mappings 
    ![ ! -e $project_name/snps ] && mkdir -p $project_name/snps

    reads_path = project_name + '/reads/all.fastq'
    contigs_path = project_name + '/genome_data/sample_gen_h1.fasta'
    contigs_path = ref_hap_path

    # Indexing and Mapping
    !minimap2 --cs -a $contigs_path $reads_path > $(pwd)/$project_name/mappings/mapping.sam
    !samtools view -Sb $(pwd)/$project_name/mappings/mapping.sam > $(pwd)/$project_name/mappings/mapping.bam
    !samtools sort $(pwd)/$project_name/mappings/mapping.bam > $(pwd)/$project_name/mappings/mapping.sorted.bam
    !samtools index $(pwd)/$project_name/mappings/mapping.sorted.bam

    # Phasing Methods
    !samtools mpileup -q 20 -Q 0 $project_name/mappings/mapping.sorted.bam | python /media/anuradhawick/data/Tools/anaconda/anaconda3/lib/python3.6/site-packages/whatshap/simple_snp_caller.py --minabs 3 --minrel 0.25 --sample sample $contigs_path | awk '($1~/#/) || ($4~/[ACGT]/ && $5~/[ACGT]/)' > $(pwd)/$project_name/snps/initial_call.vcf
    !whatshap genotype --ignore-read-groups --reference $contigs_path -o $(pwd)/$project_name/snps/genotyped.vcf $(pwd)/$project_name/snps/initial_call.vcf $project_name/mappings/mapping.sorted.bam
    !whatshap phase --ignore-read-groups --reference $contigs_path -o $(pwd)/$project_name/snps/phased.vcf $(pwd)/$project_name/snps/genotyped.vcf $(pwd)/$project_name/mappings/mapping.sorted.bam

## 2. Validating SNPs

In [5]:
# VCF parser
import vcf

def validate_snps(project_name):
    snp_truth_path = project_name + '/genome_data/diff_pos.txt'
    vcf_call_path = project_name + '/snps/initial_call.vcf'
    with open(snp_truth_path) as f:
        snp_positions = list(map(int, f.read().strip().split()))

    vcf_reader = vcf.Reader(open(vcf_call_path))
    called_pos = [int(x.POS) for x in vcf_reader]

    true_positives = set(called_pos).intersection(set(snp_positions))
    false_negatives = set(snp_positions) - set(called_pos)
    false_positives = set(called_pos) - set(snp_positions)

    print("INFO::TRUE  POS: {0}".format(len(true_positives)))
    print("INFO::FALSE POS: {0}".format(len(false_positives)))
    print("INFO::FALSE NEG: {0}".format(len(false_negatives)))
    print()

## 3. Validating Phasing Result

In [6]:
def get_hamming_dist(s1, s2):
    d = 0
    for x in range(len(s1)):
        if s1[x] != s2[x]:
            d += 1
    return d

def validate_phasing(project_name, hap_1, hap_2):
    phased_vcf_path = project_name + '/snps/phased.vcf'
    
    phased_dict = {}
    unphased_dict = {}
    phase_set_margins = {}
    
    for record in vcf.Reader(open(phased_vcf_path)):
        pos = record.POS

        # No haplotype information
        if not record.samples[0].phased:
            unphased_dict[pos - 1] = (record.REF, record.ALT)
        else:
            # phased
            phase_set = record.samples[0].data.PS
            phase_set_margins[phase_set] = pos

            a, b = record.samples[0].data.GT.split("|")
            if int(a) == 0:
                c1 = str(record.REF)
                c2 = str(record.ALT[0])
            else:
                c2 = str(record.REF)
                c1 = str(record.ALT[0])
            phased_dict[pos - 1] = (phase_set, c1, c2)

    original_haplotype_lst = list(hap_1)

    phase_margin_indexes = sorted(list(phase_set_margins.values()))
    phased_contigs = [] # array of tuples

    hap1 = list(original_haplotype_lst)
    hap2 = list(original_haplotype_lst)

    for pos, var in phased_dict.items():
        hap1[pos] = var[1]
        hap2[pos] = var[2]

    temp = 0
    for pos in phase_margin_indexes:
        hx1 = hap1[temp:pos]
        hx2 = hap2[temp:pos]
        phased_contigs.append((temp, pos, hx1, hx2))
        temp = pos

    for s, e, h1, h2 in  phased_contigs:
        seq1_part = hap_1[s:e]
        seq2_part = hap_2[s:e]
        hap1_str = ''.join(h1)
        hap2_str = ''.join(h2)

        print('INFO::Hamming distances b/w s1/h1 = {0}'.format(get_hamming_dist(seq1_part, hap1_str)))
        print('INFO::Hamming distances b/w s1/h2 = {0}'.format(get_hamming_dist(seq1_part, hap2_str)))
        print('INFO::Hamming distances b/w s2/h1 = {0}'.format(get_hamming_dist(seq2_part, hap1_str)))
        print('INFO::Hamming distances b/w s2/h2 = {0}'.format(get_hamming_dist(seq2_part, hap2_str)))

        print("=======================================")
    


# Experiment Exec 1MB Het 1%

In [45]:
project_name = 'simulated_haps_1mb_h1'

def get_seq():
    return get_haps(0.01, 1000)

hap_1, hap_2 = generate_sequences(project_name, get_seq)

generate_reads(project_name)
ref_hap_path = project_name + '/genome_data/sample_gen_h1.fasta'
snp_and_phase(project_name)
validate_snps(project_name)
validate_phasing(project_name, hap_1, hap_2)

project_name = 'simulated_haps_1mb_h1'
ref_hap_path = project_name + '/genome_data/sample_gen_h1.fasta'
snp_and_phase(project_name)
validate_snps(project_name)
validate_phasing(project_name, hap_1, hap_2)

Time for reading/generating the reference: 0:00:00.071533 h
Time for simulation of 4000 reads: 0:01:06.016984 h.
[M::mm_idx_gen::0.023*1.02] collected minimizers
[M::mm_idx_gen::0.030*1.48] sorted minimizers
[M::main::0.031*1.48] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.033*1.44] mid_occ = 3
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.035*1.42] distinct minimizers: 186691 (99.94% are singletons); average occurrences: 1.001; average spacing: 5.353
[M::worker_pipeline::7.818*2.82] mapped 4000 sequences
[M::main] Version: 2.14-r886-dirty
[M::main] CMD: minimap2 --cs -a simulated_haps_1mb/genome_data/sample_gen_h1.fasta simulated_haps_1mb/reads/all.fastq
[M::main] Real time: 7.822 sec; CPU: 22.072 sec; Peak RSS: 0.204 GB
[mpileup] 1 samples in 1 input files
running only genotyping algorithm
This is WhatsHap (genotyping) 0.18 running under Python 3.6.8
Working on 1 samples from 1 family
======== Working on chromosome 'sample

# Experiment Exec 0.2,0.2,0.2,0.2,0.2 KB alternate regions Het 1%

In [54]:
project_name = 'simulated_haps_0.2_0.2_0.2_0.2_0.2_alternate'

def get_seq():
    s11, s12 = get_haps(0.01, 200)
    s21 = s22 = createFrag(200)
    s31, s32 = get_haps(0.01, 200)
    s41 = s42 = createFrag(200)
    s51, s52 = get_haps(0.01, 200)
    
    return s11 + s21 + s31 + s41 + s51, s12 + s22 + s32 + s42 + s52

hap_1, hap_2 = generate_sequences(project_name, get_seq)
generate_reads(project_name)
ref_hap_path = project_name + '/genome_data/sample_gen_h1.fasta'
snp_and_phase(project_name)
validate_snps(project_name)
validate_phasing(project_name, hap_1, hap_2)

Time for reading/generating the reference: 0:00:00.072570 h
Time for simulation of 4000 reads: 0:01:07.011181 h.
[M::mm_idx_gen::0.025*1.03] collected minimizers
[M::mm_idx_gen::0.034*1.51] sorted minimizers
[M::main::0.034*1.51] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.036*1.47] mid_occ = 3
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.038*1.45] distinct minimizers: 186390 (99.93% are singletons); average occurrences: 1.001; average spacing: 5.361
[M::worker_pipeline::8.171*2.82] mapped 4000 sequences
[M::main] Version: 2.14-r886-dirty
[M::main] CMD: minimap2 --cs -a simulated_haps_0.2_0.2_0.2_0.2_0.2_alternate/genome_data/sample_gen_h1.fasta simulated_haps_0.2_0.2_0.2_0.2_0.2_alternate/reads/all.fastq
[M::main] Real time: 8.174 sec; CPU: 23.039 sec; Peak RSS: 0.204 GB
[mpileup] 1 samples in 1 input files
running only genotyping algorithm
This is WhatsHap (genotyping) 0.18 running under Python 3.6.8
Working on 1 samples 

In [55]:
project_name = 'simulated_haps_0.2_0.6_0.2_alternate'

def get_seq():
    s11, s12 = get_haps(0.01, 200)
    s21 = s22 = createFrag(600)
    s31, s32 = get_haps(0.01, 200)
    
    return s11 + s21 + s31, s12 + s22 + s32

hap_1, hap_2 = generate_sequences(project_name, get_seq)
generate_reads(project_name)
ref_hap_path = project_name + '/genome_data/sample_gen_h1.fasta'
snp_and_phase(project_name, ref_hap_path)
validate_snps(project_name)
validate_phasing(project_name, hap_1, hap_2)

Time for reading/generating the reference: 0:00:00.075196 h
Time for simulation of 4000 reads: 0:01:07.760454 h.
[M::mm_idx_gen::0.025*1.03] collected minimizers
[M::mm_idx_gen::0.033*1.47] sorted minimizers
[M::main::0.033*1.47] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.035*1.44] mid_occ = 3
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.037*1.42] distinct minimizers: 186784 (99.92% are singletons); average occurrences: 1.001; average spacing: 5.350
[M::worker_pipeline::7.817*2.81] mapped 4000 sequences
[M::main] Version: 2.14-r886-dirty
[M::main] CMD: minimap2 --cs -a simulated_haps_0.2_0.6_0.2_alternate/genome_data/sample_gen_h1.fasta simulated_haps_0.2_0.6_0.2_alternate/reads/all.fastq
[M::main] Real time: 7.828 sec; CPU: 21.971 sec; Peak RSS: 0.203 GB
[mpileup] 1 samples in 1 input files
running only genotyping algorithm
This is WhatsHap (genotyping) 0.18 running under Python 3.6.8
Working on 1 samples from 1 family
==